In [24]:
import psycopg2
import time
import random

# root id: postgres, password: 1234
# DB dbms_final owner: postgres (only user can create table) 
conn_string = "host='140.112.12.188' user='postgres' dbname='dbms_final' password='1234' port='5432'"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [28]:
import csv
csv_file = './patient.csv/patient.csv'
patient_site_map = {}

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        id_number = row['ID_Number']
        site_id = row['Site_ID']
        patient_site_map[id_number] = site_id
        print(patient_site_map)

In [25]:
cursor = conn.cursor()

def query_patient(patient_id, doctor_site_id):
    cursor.execute(f"SET app.current_doctor_site_id = '{doctor_site_id}';")
    start_time = time.time()
    cursor.execute("SELECT * FROM Patient WHERE ID_Number = %s AND Site_ID = %s", (patient_id, doctor_site_id))
    result = cursor.fetchall()
    end_time = time.time()
    return end_time - start_time, result

cursor.execute("DROP POLICY patient_policy ON Patient;")

cursor.execute("ALTER TABLE Patient ENABLE ROW LEVEL SECURITY;")

cursor.execute("""
    CREATE POLICY patient_policy ON Patient
    USING (EXISTS (
        SELECT 1 FROM Doctor 
        WHERE Doctor.Site_ID = Patient.Site_ID 
        AND Doctor.ID_Number = current_setting('app.current_doctor_id')
    ));
""")

query_times_same_site = []
query_times_diff_site = []

for _ in range(100): 
    print(f"The {_}st test.")
    for patient_id, patient_site_id in patient_site_map.items():
        doctor_site_id = random.choice(range(5))
        query_time, _ = query_patient(patient_id, doctor_site_id)
        if doctor_site_id == patient_site_id:
            query_times_same_site.append(query_time)
        else:
            query_times_diff_site.append(query_time)

avg_query_time_same_site = sum(query_times_same_site) / len(query_times_same_site)
avg_query_time_diff_site = sum(query_times_diff_site) / len(query_times_diff_site)

print(f"對於Site_ID和所欲訪問之Patient之Site_ID相同的doctor來說，平均query時間為: {avg_query_time_same_site} 秒")
print(f"對於Site_ID和所欲訪問之Patient之Site_ID不同的doctor來說，平均query時間為: {avg_query_time_diff_site} 秒")

cursor.close()
conn.close()

The 0st test.


KeyboardInterrupt: 

In [ ]:
def query_patient(patient_id, doctor_site_id):
    cursor.execute(f"SELECT set_config('app.site_id', '{doctor_site_id}', false);")

    start_time = time.time()
    cursor.execute("SELECT * FROM Patient WHERE ID_Number = %s AND Site_ID = %s", (patient_id, doctor_site_id))
    result = cursor.fetchall()
    end_time = time.time()
    return end_time - start_time, result

query_times_same_site = []
query_times_diff_site = []

for _ in range(100): 
    print(f"The {_}st test.")
    for patient_id, patient_site_id in patient_site_map.items():
        doctor_site_id = random.choice(range(5))
        query_time, _ = query_patient(patient_id, doctor_site_id)
        if doctor_site_id == patient_site_id:
            query_times_same_site.append(query_time)
        else:
            query_times_diff_site.append(query_time)

avg_query_time_same_site = sum(query_times_same_site) / len(query_times_same_site)
avg_query_time_diff_site = sum(query_times_diff_site) / len(query_times_diff_site)

print(f"對於Site_ID和所欲訪問之Patient之Site_ID相同的doctor來說，平均query時間為: {avg_query_time_same_site} 秒")

In [90]:
import psycopg2

# Database connection parameters
conn_string = "host='140.112.12.188' user='postgres' dbname='dbms_final' password='1234' port='5432'"

# Connect to the database
conn = psycopg2.connect(conn_string)
print("Connection established")

# Create a cursor object
cursor = conn.cursor()

# Define the SQL for creating the function with logging
create_function_sql = """
CREATE FUNCTION site_policy(row_site int8, curr_user_id int8)
RETURNS bool
AS $$
BEGIN
    RETURN (
        (SELECT site_id
        FROM doctor
        WHERE id_number = curr_user_id) 
        = row_site
        );

END;
$$
LANGUAGE plpgsql;
"""

# Define the SQL for enabling row-level security and creating the policy
enable_rls_sql = """
ALTER TABLE patient
ENABLE ROW LEVEL SECURITY;

CREATE POLICY doctor_read
ON patient FOR SELECT
USING (site_policy(site_id, 'a'));
"""

# Execute the SQL commands
try:
    # Drop policy if it exists
    cursor.execute("DROP POLICY IF EXISTS doctor_read ON patient;")
    cursor.execute("DROP FUNCTION IF EXISTS site_policy(int8, int8);")
    # Create the function
    cursor.execute(create_function_sql)
    print("Function created successfully")
    
    # Enable row-level security and create the policy
    cursor.execute(enable_rls_sql)
    conn.commit()
    print("Row-level security enabled and policy created successfully")

    cursor.execute("SELECT relrowsecurity, relforcerowsecurity FROM pg_catalog.pg_class WHERE relname = 'patient';")
    print(cursor.fetchall())
    cursor.execute("SELECT * FROM pg_policies WHERE policyname = 'doctor_read' AND tablename = 'patient';")
    print(cursor.fetchall())
    
    # Set application user ID using custom configuration        
    cursor.execute("SELECT set_config('app.current_user_id', '5', false);")
    
    # Execute the select query
    cursor.execute("SELECT * FROM patient WHERE ID_Number = '9666427225';")
    results = cursor.fetchall()
    print("Query executed successfully, results:")
    print(results)
    
    
except Exception as e:
    conn.rollback()
    print(f"An error occurred: {e}")

# Close the cursor and connection
cursor.close()
conn.close()


Connection established
Function created successfully
Row-level security enabled and policy created successfully
[(True, True)]
[('public', 'patient', 'doctor_read', 'PERMISSIVE', ['public'], 'SELECT', "site_policy((site_id)::bigint, 'a'::text)", None)]
Query executed successfully, results:
[(6, 'John Doe', '9666427225', 35, 0)]


In [91]:
import psycopg2

# Database connection parameters
conn_string = "host='140.112.12.188' user='postgres' dbname='dbms_final' password='1234' port='5432'"

# Connect to the database
conn = psycopg2.connect(conn_string)
print("Connection established")

# Create a cursor object
cursor = conn.cursor()

# Define the SQL for creating the function
create_function_sql = """
CREATE OR REPLACE FUNCTION site_policy(row_site int8, curr_user text)
RETURNS bool
AS $$
BEGIN
    RETURN (
        SELECT site_id
        FROM doctorz
        WHERE user_name = curr_user
    ) = row_site;
END;
$$
LANGUAGE plpgsql;
"""

# Define the SQL for enabling row-level security and creating the policy
enable_rls_sql = """
ALTER TABLE patient
ENABLE ROW LEVEL SECURITY;

CREATE POLICY doctor_read
ON patient FOR SELECT
USING (site_policy(site_id, current_user));
"""

# Execute the SQL commands
try:
    # Drop policy if it exists
    cursor.execute("DROP POLICY IF EXISTS patient_policy ON patient;")
    cursor.execute("DROP POLICY IF EXISTS doctor_read ON patient;")
    cursor.execute(create_function_sql)
    cursor.execute(enable_rls_sql)
    conn.commit()
    print("Function and policy created successfully")
    cursor.execute("SELECT * FROM pg_policies WHERE tablename = 'patient';")
    print(cursor.fetchall())
    cursor.execute("SELECT * FROM patient WHERE ID_Number = '9666427225';")
    results = cursor.fetchall()
    print(results  )

except Exception as e:
    conn.rollback()
    print(f"An error occurred: {e}")

# Close the cursor and connection
cursor.close()
conn.close()


Connection established
Function and policy created successfully
[('public', 'patient', 'doctor_read', 'PERMISSIVE', ['public'], 'SELECT', 'site_policy((site_id)::bigint, (CURRENT_USER)::text)', None)]
